In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Frame Bundle Geometry on Embedded Ellipsoids

In [ ]:
from src.manifolds.ellipsoid import *
M = Ellipsoid(params=np.array([1,1,1]))
#M.params.set_value(np.array([1,0.5,0.5]))
print(M)
from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M,truncate_high_order_derivatives=True)

In [ ]:
# frame bundle
from src.framebundle import FM
FM.initialize(M,do_chart_update=M.do_chart_update)

In [ ]:
# test that adapated bases D and D^* are dual
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

A = np.zeros((6,6))
for i in range(6):
    Dp1 = np.zeros(6)
    Dp1[i] = 1.
    p1 = M.from_Dstarf(u1,Dp1)
    
    for j in range(6):
        Dv1 = np.zeros(6)
        Dv1[j] = 1.
        v1 = M.from_Df(u1,Dv1)

        A[i,j] = np.dot(v1,p1)
print(A)

A = np.zeros((6,6))
for i in range(6):
    p1 = np.zeros(6)
    p1[i] = 1.
    Dp1 = M.to_Dstarf(u1,p1)
    
    for j in range(6):
        v1 = np.zeros(6)
        v1[j] = 1.
        Dv1 = M.to_Df(u1,v1)

        A[i,j] = np.dot(Dv1,Dp1)
print(A)

In [ ]:
# test that chart update preserves FM Hamiltonian
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

for i in range(6):
#     Dp1 = np.zeros(6)
#     Dp1[i] = 1.
#     p1 = M.from_Dstarf(u1,Dp1)
    p1 = np.zeros(6)
    p1[i] = 1.

    print("M.H_FM x1:",M.H_FMf(u1,p1))
    (up2,chart2) = M.chart_update_Hamiltonian_FMf(u1,p1)
    print("M.H_FM x2:",M.H_FMf((up2[0],chart2),up2[1]))

In [ ]:
# elements
x = M.coordsf([0.,0.])

# element u=(x,nu) in FM, nu being frame for T_xM
# np.linalg.cholesky(M.gsharpf(x)) gives orthonormal basis for T_xM, multiplication scales in given directions
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = (np.concatenate((x[0],nu.flatten())),x[1])

# FM covector p
v = tensor([2.,2.])
px = np.linalg.solve(nu,v) # manifold part
pu = tensor([0.,0.,0.,0.]) # frame part
p = np.concatenate([px,pu])

print("u = ", u)
print("p = ", p)

newfig()
M.plot()
M.plotx(x,v=nu)

## FM Geodesics

In [ ]:
# Hamiltionian dynamics on FM from sub-Riemannian structure <v,w>_FM=<u^-1(v),u^-1(w)>_R^2
print(M.H_FMf(u,p))

# compute FM geodesic
(us,charts) = M.Exp_Hamiltonian_FMtf(u,p)

# plot
newfig()
M.plot(rotate=(30,80))
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5),linewidth=1.5,s=50)
plt.show()

# dynamics returning both position and momentum
(ts,qps,charts) = M.Hamiltonian_dynamics_FMf(u,p)
us = qps[:,0,:]
ps = qps[:,1,:]
print("Energy: ",np.array([M.H_FMf((q,charts),p) for (q,p,charts) in zip(us,ps,charts)]))

# Development and Stochastic Development

In [ ]:
# def initialize(M,do_chart_update=None):
#     """ development and stochastic development from R^d to M """

#     t = T.scalar()
#     dgamma = T.matrix() # velocity of Euclidean curve
#     dsm = T.matrix() # derivative of Euclidean semimartingale
#     u = M.sym_FM_element()
#     d = M.dim

#     # Deterministic development
#     def ode_development(dgamma,t,u,chart):
#         u = (u,chart)
#         nu = u[0][d:].reshape((d,-1))
#         m = nu.shape[1]

#         det = T.tensordot(M.Horizontal(u)[:,0:m], dgamma, axes = [1,0])
    
#         return det
    
# #     def chart_update(t,u,chart,dgamma=None):
# #         if do_chart_update is None:
# #             return (t,u,chart)

# #         x = (u[0:d],chart)
# #         nu = u[d:].reshape((d,-1))

# #         new_chart = M.centered_chart(M.F(x))
# #         new_x = M.update_coords(x,new_chart)[0]
# #         new_nu = M.update_vector(x,new_x,new_chart,nu)

# #         return theano.ifelse.ifelse(do_chart_update(x),
# #                 (t,u,chart),
# #                 (t,T.concatenate((new_x,new_nu.flatten())),new_chart)
# #             )

#     M.development = lambda u,dgamma: integrate(ode_development,M.chart_update_FM,u[0],u[1],dgamma)
# #     M.developmentf = M.coords_function(M.development,dgamma)

#     # Stochastic development
#     def sde_development(dsm,t,u,chart):
#         u = (u,chart)
#         nu = u[0][d:].reshape((d,-1))
#         m = nu.shape[1]

#         sto = T.tensordot(M.Horizontal(u)[:,0:m], dsm, axes = [1,0])
    
#         return (T.zeros_like(sto), sto, M.Horizontal(u)[:,0:m])

#     M.sde_development = sde_development
#     M.stochastic_development = lambda u,dsm: integrate_sde(sde_development,integrator_stratonovich,
#                                                            M.chart_update_FM,u[0],u[1],dsm)
#     M.stochastic_developmentf = M.coords_function(M.stochastic_development,dsm)

In [ ]:
# development dynamics
from src.stochastics import stochastic_development
stochastic_development.initialize(M,do_chart_update=M.do_chart_update)

In [ ]:
# deterministic development

# curve in R^2
t = np.linspace(0,10,n_steps.get_value()+1)
gamma = np.vstack([[20*np.sin(t), t**2 + 2*t]]).T
dgamma = np.diff(gamma, axis = 0)

(ts,us,charts) = M.developmentf(u,dgamma)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),5))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot anti-development
plt.figure()
plt.plot(gamma[:,0],gamma[:,1])
plt.axis('equal')
plt.show()

In [ ]:
n_steps.set_value(1000)

# stochastic development
w = dWsf(M.dim.eval()) # noise / anti-development
(ts,us,charts) = M.stochastic_developmentf(u,w)

# plot with frame
newfig()
M.plot()
M.plot_path(zip(us,charts),v_steps=np.arange(0,n_steps.eval(),50))
plt.show()

# plot only trajectory
newfig()
M.plot()
M.plot_path(zip(us[:,0:M.dim.eval()],charts))
plt.show()

# plot noise / anti-development
plt.figure()
ws = np.cumsum(w,axis=0)
plt.plot(ws[:,0],ws[:,1])
plt.axis('equal')
plt.show()

n_steps.set_value(100)

# Anisotropic  Normal Distribution

In [ ]:
# plot sample data with trajectories
K = 8
obss = np.zeros((K,n_steps.eval(),M.dim.eval()))
obs_charts = np.zeros((K,n_steps.eval(),)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[:,0:M.dim.eval()]
        obs_charts[i] = charts
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plot_path(zip(obss[i],obs_charts[i]),linewidth=.5,color=colors[i])
plt.show()

In [ ]:
# sample data
K = 1024
obss = np.zeros((K,M.dim.eval()))
obs_charts = np.zeros((K,)+x[1].shape)
# srng.seed(422)
i = 0
while i < K:
    try:
        (ts,us,charts) = M.stochastic_developmentf(u,dWsf(M.dim.eval()))
        obss[i] = us[-1][0:M.dim.eval()]
        obs_charts[i] = charts[-1]
        i += 1
    except np.linalg.linalg.LinAlgError:
        pass

# plot samples
newfig()
M.plot()
M.plotx(x,v=u[0][M.dim.eval():].reshape((M.dim.eval(),-1)))
for i in range(K):
    M.plotx((obss[i],obs_charts[i]))
plt.show()

In [ ]:
# # plot estimated density, 
# newfig()
# # plotM(alpha=.4)
# # plot_sphere_density_estimate(M, np.array([M.Ff(obs) for obs in obss]),pts=100,alpha=.8,bandwidth=.15) # spherical coordinates
# plot_density_estimate(M,np.array([M.Ff((obs,chart)) for (obs,chart) in zip(obss,obs_charts)]),limits=[-3,3,-3,3],pts=500,alpha=.4,bandwidth=.15) # general ellipsoidal coordinates (note: very long computation time)
# plt.show()

# Most Probable Paths

In [ ]:
# Compute 'most probable path' (in the sense of the driving semi-martingale) between u and x2
x2 = np.array([0.5,-0.5])

# cotangent vector for the MPP:
px2 = M.Log_FM(u,x2)

# MPP from u to x2:
usv = M.Exp_Hamiltonian_FMf(u,px2).T

newfig()
ax = plt.gca(projection='3d')
M.plot(rotate=(30,-45))
M.plotFMx(usv,N_vec=5,linewidth=1.5,s=50)
xq = M.Ff(x2)
ax.scatter(xq[0],xq[1],xq[2],color='r')
plt.show()

# Horizontal Vector Fields

In [ ]:
def plotHorizontalFields(u,color='b',color_intensity=1.,linewidth=3.,prevx=None,last=True):
        xM = u[0:M.dim.eval()]
        if xM.shape[0] == M.dim.eval(): # map to manifold
            xM = M.Ff(xM)
         
        ax = plt.gca(projection='3d')
        ax.scatter(xM[0],xM[1],xM[2],color=color)

        x = u[0:M.dim.eval()]
        nu = u[M.dim.eval():].reshape((M.dim.eval(),-1))
        
        # plot frame and horizontal variation
        Hu = M.Horizontalf(u) # horizontal basis fields
        Hnu = Hu[M.dim.eval():].reshape((M.dim.eval(),nu.shape[1],nu.shape[1])) # nu part
        JFx = M.JFf(x)
        for j in range(M.dim.eval()):
            nujM = np.dot(JFx,nu[:,j])
            HnujM = np.dot(JFx,np.dot(Hnu,nu[:,j]))
            ax.quiver(xM[0],xM[1],xM[2],nujM[0],nujM[1],nujM[2], pivot='tail',
                      arrow_length_ratio = 0.15, linewidths=1,
                      color='black',normalize=True,length=np.linalg.norm(nujM))
            for k in range(nu.shape[1]):
                basep = xM + nujM
                ax.quiver(basep[0],basep[1],basep[2],
                          HnujM[0,k],HnujM[1,k],HnujM[2,k], pivot='tail',linewidths=2.,
                          color='red',normalize=True,length=0.3)

# plot horizontal fields at x = (.1,.1) (vanishes at north pole)
x = np.array([0.1,0.1])
nu = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x)))
u = np.concatenate((x,nu.flatten()))

newfig()
M.plot()
plotHorizontalFields(u)
plt.show()

In [ ]:
# def initialize(M,do_chart_update=None):
#     """ Frame Bundle geometry """
    
#     d  = M.dim
    
#     x = M.sym_element()
#     x1 = M.sym_element()
#     v = M.sym_vector()
#     nu = M.sym_frame()
    
#     u = M.sym_FM_element()
#     q = M.sym_FM_element()
#     w = M.sym_FM_vector()
#     p = M.sym_FM_covector()
    

# initialize(M)

In [ ]:
# def initialize(M,do_chart_update=None):
#     """ Frame Bundle geometry """
    
#     d  = M.dim
    
#     x = M.sym_element()
#     x1 = M.sym_element()
#     v = M.sym_vector()
#     nu = M.sym_frame()
    
#     u = M.sym_FM_element()
#     q = M.sym_FM_element()
#     w = M.sym_FM_vector()
#     p = M.sym_FM_covector()
       
#     #### Bases shifts, see e.g. Sommer Entropy 2016 sec 2.3
#     # D denotes frame adapted to the horizontal distribution
#     def to_D(u,w):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         wx = w[0:d]
#         wnu = w[d:].reshape((d,-1))        
    
#         # shift to D basis
#         Gammanu = T.tensordot(M.Gamma_g(x),nu,(2,0))
#         Dwx = wx
#         Dwnu = T.tensordot(Gammanu,wx,(0,0))+wnu

#         return T.concatenate((Dwx,Dwnu.flatten()))
#     def from_D(u,Dw):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         Dwx = Dw[0:d]
#         Dwnu = Dw[d:].reshape((d,-1))        
    
#         # shift to D basis
#         Gammanu = T.tensordot(M.Gamma_g(x),nu,(2,0))
#         wx = Dwx
#         wnu = -T.tensordot(Gammanu,Dwx,(0,0))+Dwnu

#         return T.concatenate((wx,wnu.flatten())) 
#         # corresponding dual space shifts
#     def to_Dstar(u,p):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         px = p[0:d]
#         pnu = p[d:].reshape((d,-1))        
    
#         # shift to D basis
#         Gammanu = T.tensordot(M.Gamma_g(x),nu,(2,0))
#         Dpx = px-T.tensordot(Gammanu,pnu,((1,2),(0,1)))
#         Dpnu = pnu

#         return T.concatenate((Dpx,Dpnu.flatten()))
#     def from_Dstar(u,Dp):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         Dpx = Dp[0:d]
#         Dpnu = Dp[d:].reshape((d,-1))        
    
#         # shift to D basis
#         Gammanu = T.tensordot(M.Gamma_g(x),nu,(2,0))
#         px = Dpx+T.tensordot(Gammanu,Dpnu,((1,2),(0,1)))
#         pnu = Dpnu

#         return T.concatenate((px,pnu.flatten()))
#     M.to_D = to_D
#     M.to_Df = M.coords_function(M.to_D,w)      
#     M.from_D = from_D
#     M.from_Df = M.coords_function(M.from_D,w)        
#     M.to_Dstar = to_Dstar
#     M.to_Dstarf = M.coords_function(M.to_Dstar,p)      
#     M.from_Dstar = from_Dstar
#     M.from_Dstarf = M.coords_function(M.from_Dstar,p)        
    
    
#     def Dg_FMDstar(u,Dp):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         Dpx = Dp[0:d]
#         Dpnu = Dp[d:].reshape((d,-1))
        
#         Winv = T.tensordot(nu, nu, axes = [1,1])        
#         DgDpx = T.tensordot(Winv,Dpx,(1,0))
        
#         return T.concatenate((DgDpx,T.zeros_like(Dpnu).flatten()))
#     M.Dg_FMDstar = Dg_FMDstar
#     M.Dg_FMDstarf = M.coords_function(M.Dg_FMDstar,p)    
    
#     def H_FM(u,p):
#         Dp = M.to_Dstar(u,p)
#         Dgp = Dg_FMDstar(u,Dp)
        
#         return 0.5*T.dot(Dp,Dgp)
# #     def H_FM(u,p):
# #         x = (u[0][0:d],u[1])
# #         nu = u[0][d:].reshape((d,-1))
# #         px = p[0:d]
# #         pnu = p[d:].reshape((d,-1))
        
# #         GamX = T.tensordot(M.Gamma_g(x), nu, 
# #                            axes = [2,0]).dimshuffle(0,2,1)
    
# #         Winv = T.tensordot(nu, nu, axes = [1,1])
    
# #         gij = Winv
# #         gijb = -T.tensordot(Winv, GamX, axes = [1,2])
# #         giaj = -T.tensordot(GamX, Winv, axes = [2,0])
# #         giajb = T.tensordot(T.tensordot(GamX, Winv, axes = [2,0]), 
# #                             GamX, axes = [2,2])
    
# #         pxgpx = T.dot(T.tensordot(px, gij, axes = [0,0]), px)
# #         pxgpnu = T.tensordot(T.tensordot(px, gijb, axes = [0,0]), 
# #                              pnu, axes = [[0,1],[0,1]])
# #         pnugpx = T.tensordot(T.tensordot(px, giaj, axes = [0,2]), 
# #                              pnu, axes = [[0,1],[0,1]])
# #         pnugpnu = T.tensordot(T.tensordot(giajb, pnu, axes = [[2,3],[0,1]]), 
# #                               pnu, axes = [[0,1],[0,1]])
    
# #         return 0.5*(pxgpx + pxgpnu + pnugpx + pnugpnu)
#     M.H_FM = H_FM
#     M.H_FMf = M.coords_function(M.H_FM,p)    

# initialize(M)    

In [ ]:
# def initialize(M,do_chart_update=None):
#     """ Frame Bundle geometry """
    
#     d  = M.dim

#     x = M.sym_element()
#     x1 = M.sym_element()
#     v = M.sym_vector()
#     nu = M.sym_frame()

#     def sym_FM_element():
#         """ return element of FM as concatenation (x,nu) flattened """
#         return T.vector()
#     def sym_FM_vector():
#         """ vector in TFM """
#         return T.vector()
#     def sym_FM_covector():
#         """ covector in T^*FM """
#         return T.vector()
#     M.sym_FM_element = sym_FM_element
#     M.sym_FM_vector = sym_FM_vector
#     M.sym_FM_covector = sym_FM_covector  

#     u = M.sym_FM_element()
#     q = M.sym_FM_element()
#     p = M.sym_FM_covector()

#     ##### Cometric matrix:
#     def g_FMsharp(u):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,d))#.reshape((d,M.m))
#         GamX = T.tensordot(M.Gamma_g(x), nu, axes = [2,0]).dimshuffle(0,2,1)
    
#         delta = T.eye(nu.shape[0],nu.shape[1])
#         W = T.tensordot(nu,  nu,  axes = [1,1]) + lambdag0*M.g(x)
    
#         gij = W
#         gijb = -T.tensordot(W, GamX, axes = [1,2])
#         giaj = -T.tensordot(GamX, W, axes = [2,0])
#         giajb = T.tensordot(T.tensordot(GamX, W, axes = [2,0]), 
#                             GamX, axes = [2,2])

#         return gij,gijb,giaj,giajb

#     ##### Hamiltonian on FM based on the pseudo metric tensor: 
#     lambdag0 = 0

#     def H_FM(u,p):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
#         px = p[0:d]
#         pnu = p[d:].reshape((d,-1))
        
#         GamX = T.tensordot(M.Gamma_g(x), nu, 
#                            axes = [2,0]).dimshuffle(0,2,1)
    
#         delta = T.eye(nu.shape[0],nu.shape[1])
#         W = T.tensordot(nu, nu, axes = [1,1]) + lambdag0*M.g(x)
    
#         gij = W
#         gijb = -T.tensordot(W, GamX, axes = [1,2])
#         giaj = -T.tensordot(GamX, W, axes = [2,0])
#         giajb = T.tensordot(T.tensordot(GamX, W, axes = [2,0]), 
#                             GamX, axes = [2,2])
    
#         pxgpx = T.dot(T.tensordot(px, gij, axes = [0,0]), px)
#         pxgpnu = T.tensordot(T.tensordot(px, gijb, axes = [0,0]), 
#                              pnu, axes = [[0,1],[0,1]])
#         pnugpx = T.tensordot(T.tensordot(px, giaj, axes = [0,2]), 
#                              pnu, axes = [[0,1],[0,1]])
#         pnugpnu = T.tensordot(T.tensordot(giajb, pnu, axes = [[2,3],[0,1]]), 
#                               pnu, axes = [[0,1],[0,1]])
    
#         return 0.5*(pxgpx + pxgpnu + pnugpx + pnugpnu)

#     M.H_FM = H_FM
#     M.H_FMf = M.coords_function(M.H_FM,p)

#     ##### Evolution equations:
#     dq = lambda q,p: T.grad(M.H_FM(q,p),p)
#     dp = lambda q,p: -T.grad(M.H_FM(q,p),q[0])

#     def ode_Hamiltonian_FM(t,x,chart): # Evolution equations at (p,q).
#         dqt = dq((x[0],chart),x[1])
#         dpt = dp((x[0],chart),x[1])
#         return T.stack((dqt,dpt))

#     def chart_update_FM(t,up,chart):
#         if do_chart_update is None:
#             return (t,u,chart)
        
#         u = up[0]
#         p = up[1]

#         x = (u[0:d],chart)
#         nu = u[d:].reshape((d,-1))
#         px = (p[0:d],chart)
#         pnu = p[d:].reshape((d,-1))

#         new_chart = M.centered_chart(M.F(x))
#         new_x = M.update_coords(x,new_chart)[0]
#         new_nu = M.update_vector(x,new_x,new_chart,nu)
#         new_px = M.update_coords(x,new_chart)[0] # fix this
#         new_pnu = M.update_vector(x,new_x,new_chart,pnu) # fix this
        
#         return theano.ifelse.ifelse(do_chart_update(x),
#                 (t,up,chart),
#                 (t,T.stack((T.concatenate((new_x,new_nu.flatten())),T.concatenate((new_px,new_pnu.flatten())))),new_chart)
#             )
#     M.chart_update_FM = chart_update_FM

#     M.Hamiltonian_dynamics_FM = lambda q,p: integrate(ode_Hamiltonian_FM,chart_update_FM,T.stack((q[0],p)),q[1])
#     M.Hamiltonian_dynamics_FMf = M.coords_function(M.Hamiltonian_dynamics_FM,p)

#     ## Geodesic
#     def Exp_Hamiltonian_FM(u,p):
#         curve = M.Hamiltonian_dynamics_FM(u,p)
#         u = curve[1][-1,0]
#         chart = curve[2][-1]
#         return(u,chart)
#     M.Exp_Hamiltonian_FM = Exp_Hamiltonian_FM
#     M.Exp_Hamiltonian_FMf = M.coords_function(M.Exp_Hamiltonian_FM,p)
#     def Exp_Hamiltonian_FMt(u,p):
#         curve = M.Hamiltonian_dynamics_FM(u,p)
#         u = curve[1][:,0].dimshuffle((1,0))
#         chart = curve[2]
#         return(u,chart)
#     M.Exp_Hamiltonian_FMt = Exp_Hamiltonian_FMt
#     M.Exp_Hamiltonian_FMtf = M.coords_function(M.Exp_Hamiltonian_FMt,p)

#     # Most probable path for the driving semi-martingale
#     M.loss = lambda u,x,p: 1./d*T.sum((M.Exp_Hamiltonian_FM(u,p)[0][0:d] - x[0:d])**2)
#     M.lossf = M.coords_function(M.loss,x,p)

#     def Log_FM(u,x):
#         def fopts(p):
#             y = M.lossf(u,x,p)
#             return y

#         res = minimize(fopts, np.zeros(u.shape), 
#                        method='CG', jac=False, options={'disp': False, 
#                                                         'maxiter': 50})
#         return res.x
#     M.Log_FM = Log_FM

#     ##### Horizontal vector fields:
#     def Horizontal(u):
#         x = (u[0][0:d],u[1])
#         nu = u[0][d:].reshape((d,-1))
    
#         # Contribution from the coordinate basis for x: 
#         dx = nu
#         # Contribution from the basis for Xa:
#         dnu = -T.tensordot(nu, T.tensordot(nu, M.Gamma_g(x), axes = [0,2]), axes = [0,2])

#         dnuv = dnu.reshape((nu.shape[1],dnu.shape[1]*dnu.shape[2]))

#         return T.concatenate([dx,dnuv.T], axis = 0)
#     M.Horizontal = Horizontal
#     M.Horizontalf = M.coords_function(M.Horizontal)
    

In [ ]:
# test that chart update preserves FM Hamiltonian
x1 = M.coordsf([1.2,0.])
nu1 = np.dot(np.diag((.5,1.)),np.linalg.cholesky(M.gsharpf(x1)))
u1 = (np.concatenate((x1[0],nu1.flatten())),x1[1])

# x2 = M.update_coordsf(x1,M.centered_chartf(M.Ff(x1)))
# nu2 = np.stack((M.update_vectorf(x1,x2,nu1[:,0]),
#                 M.update_vectorf(x1,x2,nu1[:,1])),axis=1)
# u2 = (np.concatenate((x2[0],nu2.flatten())),x2[1])

# print("x1:\n",x1,"\nx2:\n",x2)
# print("u1:\n",u1,"\nu2:\n",u2)
# print("nu1^2:\n",np.dot(nu1,nu1.T),"\nnu2^2:\n",np.dot(nu2,nu2.T))

for i in range(6):
#     Dp1 = np.zeros(6)
#     Dp1[i] = 1.
#     p1 = M.from_Dstarf(u1,Dp1)
    p1 = np.zeros(6)
    p1[i] = 1.
    Dp1 = M.to_Dstarf(u1,p1)
#     print("i: ",p1,Dp1,M.to_Dstarf(u1,p1))

    print("M.H_FM x1:",M.H_FMf(u1,p1))
#     print("M.Dg_FMDstar:",M.Dg_FMDstarf(u1,p1))

#     Dp2x = M.update_covectorf(x1,x2,Dp1[0:M.dim.eval()])
#     Dp2nu = np.stack((M.update_covectorf(x1,x2,Dp1[M.dim.eval():].reshape((M.dim.eval(),-1))[:,0]),
#                       M.update_covectorf(x1,x2,Dp1[M.dim.eval():].reshape((M.dim.eval(),-1))[:,1])),axis=1)
#     Dp2 = np.concatenate((Dp2x,Dp2nu.flatten()))
# #     print(Dp2)
#     p2 = M.from_Dstarf(u2,Dp2)
    (up2,chart2) = M.chart_update_Hamiltonian_FMf(u1,p1)
    print("M.H_FM x2:",M.H_FMf((up2[0],chart2),up2[1]))

In [ ]:
# def initialize(M,do_chart_update=None):
#     """ Frame Bundle geometry """
    
#     d  = M.dim

#     x = M.sym_element()
#     x1 = M.sym_element()
#     v = M.sym_vector()
#     nu = M.sym_frame()

#     def sym_FM_element():
#         """ return element of FM as concatenation (x,nu) flattened """
#         return T.vector()
#     def sym_FM_vector():
#         """ vector in TFM """
#         return T.vector()
#     def sym_FM_covector():
#         """ covector in T^*FM """
#         return T.vector()
#     M.sym_FM_element = sym_FM_element
#     M.sym_FM_vector = sym_FM_vector
#     M.sym_FM_covector = sym_FM_covector  

#     u = M.sym_FM_element()
#     q = M.sym_FM_element()
#     w = M.sym_FM_vector()    
#     p = M.sym_FM_covector()

#     ##### Evolution equations:
#     dq = lambda q,p: T.grad(M.H_FM(q,p),p)
#     dp = lambda q,p: -T.grad(M.H_FM(q,p),q[0])

#     def ode_Hamiltonian_FM(t,x,chart): # Evolution equations at (p,q).
#         dqt = dq((x[0],chart),x[1])
#         dpt = dp((x[0],chart),x[1])
#         return T.stack((dqt,dpt))

#     def chart_update_Hamiltonian_FM(t,up,chart):
#         if do_chart_update is None:
#             return (t,up,chart)
        
#         u = up[0]
#         p = up[1]
#         Dp = M.to_Dstar((u,chart),p)

#         x = (u[0:d],chart)
#         nu = u[d:].reshape((d,-1))
#         Dpx = Dp[0:d]
#         Dpnu = Dp[d:].reshape((d,-1))

#         new_chart = M.centered_chart(M.F(x))
#         new_x = M.update_coords(x,new_chart)[0]
#         new_nu = M.update_vector(x,new_x,new_chart,nu)
#         new_u = T.concatenate((new_x,new_nu.flatten()))
#         new_Dpx = M.update_covector(x,new_x,new_chart,Dpx)
#         new_Dpnu = M.update_covector(x,new_x,new_chart,Dpnu)
#         new_Dp = T.concatenate((new_Dpx,new_Dpnu.flatten()))
        
#         return theano.ifelse.ifelse(do_chart_update(x),
#                 (t,up,chart),(t,T.stack((new_u,
#                                          M.from_Dstar((new_u,new_chart),new_Dp))),
#                               new_chart)
#             )
#     M.chart_update_Hamiltonian_FM = chart_update_Hamiltonian_FM
#     M.chart_update_Hamiltonian_FMf = M.coords_function(lambda u,p: M.chart_update_Hamiltonian_FM(0.,(u[0],p),u[1])[1:],p)

#     M.Hamiltonian_dynamics_FM = lambda q,p: integrate(ode_Hamiltonian_FM,chart_update_Hamiltonian_FM,T.stack((q[0],p)),q[1])
# #     M.Hamiltonian_dynamics_FMf = M.coords_function(M.Hamiltonian_dynamics_FM,p)

#     ## Geodesic
#     def Exp_Hamiltonian_FM(u,p):
#         curve = M.Hamiltonian_dynamics_FM(u,p)
#         u = curve[1][-1,0]
#         chart = curve[2][-1]
#         return(u,chart)
#     M.Exp_Hamiltonian_FM = Exp_Hamiltonian_FM
# #     M.Exp_Hamiltonian_FMf = M.coords_function(M.Exp_Hamiltonian_FM,p)
#     def Exp_Hamiltonian_FMt(u,p):
#         curve = M.Hamiltonian_dynamics_FM(u,p)
#         u = curve[1][:,0].dimshuffle((1,0))
#         chart = curve[2]
#         return(u,chart)
#     M.Exp_Hamiltonian_FMt = Exp_Hamiltonian_FMt
# #     M.Exp_Hamiltonian_FMtf = M.coords_function(M.Exp_Hamiltonian_FMt,p)

#     # Most probable path for the driving semi-martingale
#     M.loss = lambda u,x,p: 1./d*T.sum((M.Exp_Hamiltonian_FM(u,p)[0][0:d] - x[0:d])**2)
# #     M.lossf = M.coords_function(M.loss,x,p)
# initialize(M,do_chart_update=M.do_chart_update)